In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\aamer\\OneDrive\\Desktop\\Projects\\train2\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\aamer\\OneDrive\\Desktop\\Projects\\train2'

In [5]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen= True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    test_data_info : Path
    all_params : dict
    params_image_size : list 
    params_batch_size : int 

In [7]:
from CNNproject.constants import *
from CNNproject.utils.common import read_yaml, create_directories, save_json, load_bin


In [ ]:
class ConfigurationManager: 
    def __init__ (self, config_file_path =CONFIG_YAML, params_file_path= PARMAS_YAML ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config (self) -> EvaluationConfig : 
        eval_config = EvaluationConfig (
            path_of_model = Path("artifacts/training/model.h5"),
            training_data = Path("artifacts/data_ingestion/Chicken-fecal-images"),
            test_data_info = Path("artifacts/test_data/test_data.pkl"),
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE, 
            params_batch_size= self.params.BATCH_SIZE
        )
        
        return eval_config

In [9]:
from CNNproject.utils.common import load_bin

class Evaluation: 
    def __init__(self, config: EvaluationConfig): 
        self.config = config 
        tf.config.run_functions_eagerly(True)

    def _valid_generator(self):
        file_path = self.config.test_data_info
        test_data_info = load_bin(file_path)  # Load test data

        dataflow_kwargs = dict(                                                 
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Step 1: Create Test Generator
        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255, validation_split=0.25
        )
        
        test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.training_data, 
            subset="validation",  # This will be the test set
            shuffle=False,  
            **dataflow_kwargs
        )

        # Step 2: Restore Attributes from Saved Test Data
        test_generator.filenames = test_data_info.get('filenames', test_generator.filenames)
        test_generator.classes = test_data_info.get('classes', test_generator.classes)
        test_generator.samples = test_data_info.get('samples', test_generator.samples)
        test_generator.batch_size = test_data_info.get('batch_size', test_generator.batch_size)

        return test_generator
    

    @staticmethod 
    def load_model (path:Path) -> tf.keras.Model : 
        return tf.keras.models.load_model(path)
    

    def evaluation(self) :
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator = self._valid_generator()
        self.score = self.model.evaluate(self._valid_generator)

    def save_score(self) :
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data= scores)
     

In [10]:
try : 
    config= ConfigurationManager()
    test_config = config.get_evaluation_config()
    evaluation = Evaluation(config=test_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e : 
    raise e 


[2025-02-10 16:43:08,846: INFO: common: yaml file config\config.yaml loadded successfully]
[2025-02-10 16:43:08,846: INFO: common: yaml file params.yaml loadded successfully]
[2025-02-10 16:43:08,846: INFO: common: Created directory at: artifacts]
[2025-02-10 16:43:08,846: INFO: common: Created directory at: artifacts\test_data\test_data.pkl]


PermissionError: [Errno 13] Permission denied: 'artifacts\\test_data\\test_data.pkl'